In [1]:
import re
import pandas as pd

In [103]:
mean_tmp=pd.read_csv("climate-data/mean-precip-US-year-states.csv")

mean_tmp

,Date-state,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948-AR,3.302,5.765,5.14,3.125,5.062,5.4239999999999995,3.833,3.8539999999999996,0.871,2.532,5.245,4.206
1,1949-AR,10.354000000000001,3.9939999999999998,5.4079999999999995,2.235,4.763,5.329,4.908,2.7889999999999997,2.939,8.491,0.358,4.859
2,1950-AR,9.714,6.022,3.8689999999999998,3.055,7.957000000000001,4.029,6.196,6.593,5.192,1.911,2.299,0.929
3,1951-AR,4.876,5.252000000000001,2.209,4.473,2.093,7.102,5.765,2.641,4.826,4.428,5.684,3.233
4,1952-AR,3.72,3.5180000000000002,5.642,6.0489999999999995,4.215,0.511,2.676,3.565,1.768,0.829,7.834,3.587
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2016-WV,2.6439999999999997,3.497,2.294,2.622,6.007999999999999,6.111000000000001,4.713,3.4730000000000003,3.157,3.098,1.7109999999999999,4.229
3500,2017-WV,4.5760000000000005,1.571,4.003,4.3260000000000005,6.462999999999999,4.315,5.948,3.286,1.746,5.290,2.205,1.606
3501,2018-WV,2.8280000000000003,6.2589999999999995,3.6710000000000003,4.791,6.516,6.579,4.629,5.996,9.484,3.781,5.044,4.882
3502,2019-WV,3.734,5.337000000000001,2.6830000000000003,3.965,5.405,4.783,4.665,3.5980000000000003,0.720,5.220,2.39,4.286


In [96]:
mean_tmp.to_csv("climate-data/check.csv")

In [94]:
pd.to_numeric(mean_tmp["mean_tmp"])


ValueError: Unable to parse string "4.642   10.157" at position 2304

In [86]:
dic=pd.read_csv("climate-data/state_region_dic.csv",index_col=0, squeeze=True).to_dict()
mean_tmp['HHS_region']=mean_tmp['state'].map(dic)
mean_tmp=mean_tmp.dropna()
mean_tmp

,Date-state,month,mean_tmp,year,state,HHS_region
0,1948-AR,Jan,3.302,1948,AR,region6
1,1949-AR,Jan,10.354000000000001,1949,AR,region6
2,1950-AR,Jan,9.714,1950,AR,region6
3,1951-AR,Jan,4.876,1951,AR,region6
4,1952-AR,Jan,3.72,1952,AR,region6
...,...,...,...,...,...,...
42043,2016-WV,Dec,4.229,2016,WV,region3
42044,2017-WV,Dec,1.606,2017,WV,region3
42045,2018-WV,Dec,4.882,2018,WV,region3
42046,2019-WV,Dec,4.286,2019,WV,region3


In [91]:
mean_tmp2=mean_tmp[['HHS_region',"mean_tmp"]]
mean_tmp2.to_csv("climate-data/check.csv")

In [90]:
pd.to_numeric(mean_tmp2["mean_tmp"])

ValueError: Unable to parse string "4.642   10.157" at position 2304

In [70]:
mean_tmp.to_csv("climate-data/mean-temp-US-year-region.csv")

In [106]:
## wrap to function
def region_season(file,param):
    df=pd.read_csv(file)
    df=pd.melt(df,id_vars=['Date-state'],var_name='month', value_name=param)
    df[['year','state']] = df['Date-state'].str.split('-',expand=True)
    df.loc[df['month'].isin(["Sep","Oct","Nov","Dec"]), 'season'] = pd.to_numeric(df["year"])+1
    df.loc[df['month'].isin(["Jan","Feb","Mar","Apr"]), 'season'] = pd.to_numeric(df["year"])
    dic=pd.read_csv("climate-data/state_region_dic.csv",index_col=0, squeeze=True).to_dict()
    df['HHS_region']=df['state'].map(dic)
    df=df.dropna()
    result=df[['season','HHS_region',param]].groupby(["HHS_region","season"]).mean()
    output=file.replace("year-states", "season-region")
    result.to_csv(output)
    print("output file is written in " + output)

    

In [107]:
region_season("climate-data/mean-temp-US-year-states.csv","mean_temp")

output file is written in climate-data/mean-temp-US-season-region.csv


In [83]:
region_season("climate-data/mean-precip-US-year-states2.csv","mean_precip")

output file is written in climate-data/mean-precip-US-season-region2.csv


/var/folders/5l/1xy2811j5n1dzz9s802679j80000gr/T/ipykernel_29783/3588393639.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  region_season("climate-data/mean-precip-US-year-states2.csv","mean_precip")


In [58]:
region_season("climate-data/mean-palmerdrought-US-year-states.csv","mean_palmerdrought")

output file is written in climate-data/mean-palmerdrought-US-season-region.csv


In [97]:
df=pd.read_csv("climate-data/nclimdiv-averages-flu-sep-april.csv")

In [98]:
df[['year','state']] = df['Date-state'].str.split('-',expand=True)
dic=pd.read_csv("climate-data/state_region_dic.csv",index_col=0, squeeze=True).to_dict()
df['HHS_region']=df['state'].map(dic)
df

,Date-state,Mean-Temp-F,Mean-Precip-Inches,Mean-Palmer-Drought-Index,Mean-Tmax,Mean-Tmin,Mean-HDD,Mean-CDD,year,state,HHS_region
0,1949-AL,56.851585,50.351605,1.805669,68.338272,45.360695,293.826477,46.947536,1949,AL,region4
1,1950-AL,57.693054,29.043085,-0.828921,69.425453,45.952122,284.853760,64.579834,1950,AL,region4
2,1951-AL,54.733738,33.621758,-0.253363,66.701805,42.775074,359.340729,48.863693,1951,AL,region4
3,1952-AL,56.847130,36.974400,-0.864070,68.850548,44.848587,309.434082,63.477402,1952,AL,region4
4,1953-AL,55.029152,38.207603,-0.703423,67.510284,42.539639,341.090729,39.683845,1953,AL,region4
...,...,...,...,...,...,...,...,...,...,...,...
3451,2016-WY,35.113258,12.967778,-0.775334,46.802826,23.432959,909.137756,4.423189,2016,WY,region8
3452,2017-WY,33.684723,21.086916,1.304545,45.179077,22.171421,949.881592,1.607201,2017,WY,region8
3453,2018-WY,32.333508,16.162271,1.320447,44.048126,20.623198,987.167480,1.093813,2018,WY,region8
3454,2019-WY,30.861584,13.160381,0.511788,42.491730,19.237461,1032.609010,1.942501,2019,WY,region8


In [109]:
mean_sum=df[['year','HHS_region','Mean-Temp-F','Mean-Precip-Inches','Mean-Palmer-Drought-Index','Mean-Tmax','Mean-Tmin','Mean-HDD','Mean-CDD']].groupby(["HHS_region","year"]).mean()

In [110]:
mean_sum.to_csv("climate-data/nclimdiv-averages-flu-sep-april_HHSregion.csv")